# Series de tiempo
Un concepto fundamental en economía, econometría y estadística es el de las series de tiempo, las cuales se definen como un conjunto de observaciones de un fenómeno ordenados en el tiempo, donde este ultimo puede ser periódico o no, lo cual estará dado por la posibilidad o interés de tomar la medida.

En este ramo, se estudiarán los conceptos básicos del manejo de series de tiempo en la primera parte del ramo, para luego estudiar algunos modelos útiles.
En términos de manejo de las series veremos:

+ Dataframes y Series en pandas
+ Índices
+ Fechas
+ formato ISO 8601
+ slices
+ Selección de datetimes parcial
+ Llenado de valores perdidos (backward fill, forward fill, interpolación
+ Medias móviles
+ Visualización de series de tiempo

In [36]:
# Se importan las librerías principales
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

# Se cambiar el directorio de trabajo para que encuentre los archivos en caso de que no lo haga
print("Actual directorio:", os.getcwd())  
# os.chdir("pandas Foundations")
# print("Actual directorio:", os.getcwd())

Actual directorio: c:\Users\ggajardo\Documents\cursos_python\pandas Foundations


In [35]:
# Se lee el 
aapl = pd.read_csv("AAPL.csv")
print(aapl.head(2))

Date        Open        High        Low       Close  Adj Close  \
0  2014-09-29  100.589996  100.690002  98.040001   99.620003  93.514290   
1  2014-10-06   99.949997  102.379997  98.309998  100.730003  94.556244   

      Volume  
0  142718700  
1  280258200  


## Índices y fechas
En el caso de querer trabajar con series de tiempo, es necesario saber como leer los archivos de forma apropiada, esto debido a que resulta muy útil trabajar con índices y no siempre pandas trabaja con ellos.
Por ello, acá se muestran tres formas de leer un mismo archivo, y con df.dtypes es posible observar la diferencia que se genera.

+ Leer el dataframe sin especificar nada, cada columna es una variable (df1). Date es por defecto tipo object
+ Indicando parse_dates, que indica claramente cual es la columna que tiene la componente de tiempo (df2). Date es de tipo datetime64[ns] (formato general para fechas en Python)
+ Indicado lo anterior, ademas de definir la fecha como índice (df23). Se observa que ya no aparece la fecha con .dtypes, debido a que ahora es un índice. Esta es la forma de leer archivos de series de tiempo.

In [0]:
filename = "AAPL.csv"
df1 = pd.read_csv(filename)
print(df1.head())
print(df1.dtypes)

In [0]:
df2 = pd.read_csv(filename, parse_dates=['Date'])
print(df2.head())
print(df2.dtypes)

In [0]:
df3 = pd.read_csv(filename, index_col='Date', parse_dates=True)
print(df3.head())
print(df3.dtypes)

En otros casos, uno quiere construir un dataframe para análisis de series de tiempo a partir de datos que uno ha procesado, en ese caso, la función .to_datetime() permite obtener lo visto anteriormente.
Para utilizar esta función es necesario conocer el formato a utilizar, y asumiremos que tenemos las series de Precio de apertura y fechas asociadas en dos listas de un proceso anterior.

In [0]:
# Se define el formato, se supone que lo conocemos
formato_fecha = '%Y-%m-%d'

# Esta parte es solo para ejemplificar
lista_fechas = df1['Date'].tolist()
lista_apertura = df1['Open'].tolist()

# Se pasa la lista de fechas al formato datetime
fechas_datetimes = pd.to_datetime(lista_fechas, format = formato_fecha) 
print(fechas_datetimes)

# Se genera la serie de tiempo
serie_temporal = pd.Series(lista_apertura, index = fechas_datetimes)
serie_temporal.head()

Con lo anterior es posible construir una Serie en pandas, que se puede entender como una columna de un DataFrame con un index, de hecho, todas las funciones para sacar estadísticas que vimos para dataframes aplican para series (describe, count, max, quantile, etc), incluso los gráficos.

In [0]:
# Ejemplos de funciones
print(serie_temporal.describe())
print(serie_temporal.count())
print(serie_temporal.quantile(0.25))

# Gráfico simple
serie_temporal.plot()
plt.show()

## Leer y cortar períodos
Las series de pandas poseen lo que se conoce como indexado parcial, es decir, que uno puede entregar solo una porción del datetime y pandas entregará el periodo especificado. A continuación se especificarán los siguientes períodos y se calcularán sus estadísticas.

+ Datos solo de 2016
+ 2016 a 2018
+ Solo del 4 de febrero de 2016
+ Del 4 de febrero de 2016 al 4 de febrero de 2018
+ Solo de marzo de 2016
+ De marzo a agosto de 2017

In [0]:
# Datos solo de 2016
s1 = serie_temporal.loc['2016']
print(s1.describe())

# 2016 a 2018
s2 = serie_temporal.loc['2016':'2018']
print(s2.describe())

# Solo del 4 de enero de 2016
s3 = serie_temporal.loc['2016-01-04']
print(s3)

# Del 4 de febrero de 2016 al 4 de febrero de 2018
s4 = serie_temporal.loc['2016-02-04':'2018-02-04']
print(s4.describe())

# Solo de marzo de 2016
s5 = serie_temporal.loc['2016-03']
print(s5.describe())

# De marzo a agosto de 2017
s6 = serie_temporal.loc['2017-03':'2017-08']
print(s6.describe())